In [5]:
from PIL import Image
from src.color import Color
from src.utils import remove_background
from src.utils import INPUT_PATH, OUTPUT_PATH, remove_background
from os import listdir
from os.path import isfile, join
# from octree_quantizer import OctreeQuantizer

In [6]:
REMOVE_BACKGROUND = True

## Remove background from images

In [7]:
# Remove backgroud from image
if (REMOVE_BACKGROUND):
    files = [f for f in listdir(INPUT_PATH) if isfile(join(INPUT_PATH, f))]

    for idx,img in enumerate(files):
        remove_background(img)


## Load image to octree

In [8]:
image = Image.open('./dataset/no_bg/1.png')
pixels = image.load()
width, height = image.size

# add colors to the octree
for j in range(height):
    for i in range(width):
        Color(*pixels[i, j])